<a href="https://colab.research.google.com/github/quinbez/The-Transformer-Architecture/blob/main/Transformer_Architecture_With_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Input Embedding**

In [25]:
import torch
import torch.nn as nn
import math

In [26]:
class InputEmbeddings(nn.Module):
  def __init__(self, d_model: int, vocab_size: int):
    super().__init__()
    self.d_model = d_model
    self.vocab_size = vocab_size
    self.embedding = nn.Embedding(vocab_size, d_model)

  def forward(self, x):
    x = self.embedding(x)
    x = x * math.sqrt(self.d_model)
    return x

### **Positional Encoding**

In [27]:
class PositionalEncoding(nn.Module):
  def __init__(self, d_model, seq_len, dropout: float)->None:
    super().__init__()
    self.d_model = d_model
    self.seq_len = seq_len
    self.dropout = nn.Dropout(dropout)

    pe = torch.zeros(seq_len, d_model)
    position = torch.arange(0, seq_len, dtype = float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * -math.log(10000.0 / d_model))

    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)

    pe = pe.unsqueeze(0)
    self.register_buffer('pe', pe)

  def forward(self, x):
    x = x + (self.pe[:, :x.shape[1], :])
    return self.dropout(x)

### **Layer Normalization**

In [28]:
class LayerNormalization(nn.Module):
  def __init__(self, eps: float = 10**-6)->None:
    super().__init__()
    self.eps = eps
    self.alpha = nn.Parameter(torch.ones(1))
    self.bias = nn.Parameter(torch.ones(1))

  def forward(self,x):
    mean = x.mean(dim = -1, keepdim = True)
    std = x.std(dim = -1, keepdim = True)
    return self.alpha * (x - mean) / (std + self.eps) + self.bias

### **Feed-Forward Network**

In [29]:
class FeedForwardBlock(nn.Module):
  def __init__(self, d_model:int, d_ff:int, dropout:float):
    super().__init__()
    self.linear_1 = nn.Linear(d_model, d_ff)
    self.dropout = nn.Dropout(dropout)
    self.linear_2 = nn.Linear(d_ff, d_model)

  def forward(self, x):
    x = torch.relu(self.linear_1(x))
    x = self.dropout(x)
    x = self.linear_2(x)
    return x

### **Multi-Head Attention**

In [30]:
class MultiHeadAttentionBlock(nn.Module):
  def __init__(self, d_model:int, h:int, dropout:float):
    super().__init__()
    self.d_model = d_model
    self.h = h

    assert d_model % h == 0, "d_model must be divisible by h"
    self.d_k = d_model // h

    self.w_q = nn.Linear(d_model, d_model)
    self.w_k = nn.Linear(d_model, d_model)
    self.w_v = nn.Linear(d_model, d_model)
    self.w_o = nn.Linear(d_model, d_model)

    self.dropout = nn.Dropout(dropout)

  @staticmethod
  def attention(query, key, value, mask, dropout:nn.Dropout):
    d_k = query.shape[-1]     # The last dimension of query, key, and value
    attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)

    if mask is not None:
      attention_scores.masked_fill_(mask == 0, -1e9)
    attention_scores = attention_scores.softmax(dim = -1)

    if dropout is not None:
      attention_scores = dropout(attention_scores)

    return (attention_scores @ value), attention_scores
  def forward(self, q, k, v, mask):
    query = self.w_q(q)
    key = self.w_k(k)
    value = self.w_v(v)

    # (Batch, seq_len, d_model) --> (Batch, seq_len, h, d_k) --> (Batch, h, seq_len, d_k)
    query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1,2)
    key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1,2)
    value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1,2)

    x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, self.dropout)

    x = x.transpose(1,2).contiguous().view(x.shape[0], -1, self.h * self.d_k)
    return self.w_o(x)

### **Residual Connection**

In [31]:
class ResidualConnection(nn.Module):
  def __init__(self, dropout: float)->None:
    super().__init__()
    self.dropout = nn.Dropout(dropout)
    self.norm = LayerNormalization()

  def forward(self, x, sublayer):
    y = self.norm(x)
    y = sublayer(y)
    y = self.dropout(y)
    return x + y


### **Encoder Block**

In [32]:
class EncoderBlock(nn.Module):
  def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout:float)->None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

  def forward(self, x, src_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
    x = self.residual_connections[1](x, self.feed_forward_block)
    return x

In [33]:
class Encoder(nn.Module):
  def __init__(self, layers:nn.ModuleList)->None:
    super().__init__()
    self.layers = layers
    self.norm = LayerNormalization()

  def forward(self, x, mask):
    for layer in self.layers:
      x = layer(x, mask)
    return self.norm(x)

### **Decoder Block**

In [34]:
class DecoderBlock(nn.Module):
  def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout:float)->None:
    super().__init__()
    self.self_attention_block = self_attention_block
    self.cross_attention_block = cross_attention_block
    self.feed_forward_block = feed_forward_block
    self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, tgt_mask))
    x = self.residual_connections[1](x, lambda x: self.self_attention_block(x, encoder_output, encoder_output, src_mask))
    x = self.residual_connections[2](x, lambda x: self.feed_forward_block)
    return x

In [35]:
class Decoder(nn.Module):
  def __init__(self, layers: nn.ModuleList)->None:
    super().__init__()
    self.layers = layers
    self.norm  = LayerNormalization()

  def forward(self, x, encoder_output, src_mask, tgt_mask):
    for layer in self.layers:
      x = layer(x, encoder_output, src_mask, tgt_mask)
    return self.norm(x)

### **Projection Layer**

In [36]:
class ProjectionLayer(nn.Module):
  def __init__(self, d_model:int, vocab_size:int)->None:
    super().__init__()
    self.proj = nn.Linear(d_model, vocab_size)

  def forward(self, x):
    x = self.proj(x)
    x = torch.log_softmax(x, dim = -1)
    return x

### **Transformer Block**

In [37]:
class Transformer(nn.Module):
  def __init__(self, encoder:Encoder, decoder:Decoder, src_embed:InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer )->None:
    super().__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.src_embed = src_embed
    self.tgt_embed = tgt_embed
    self.src_pos = src_pos
    self.tgt_pos = tgt_pos
    self.projection_layer = projection_layer

  def encode(self, src, src_mask):
    src = self.src_embed(src)
    src = self.src_pos(src)
    return self.encoder(src, src_mask)

  def decode(self, encoder_output, src_mask, tgt, tgt_mask):
    tgt = self.tgt_embed(tgt)
    tgt = self.tgt_pos(tgt)
    return self.decode(tgt, encoder_output, src_mask, tgt_mask)

  def project(self, x):
    return self.projection_layer(x)

### **Building & Initializing Transformer**

In [38]:
def build_transformer(src_vocab_size:int, tgt_vocab_size:int, src_seq_len:int, tgt_seq_len:int, d_model:int = 512, N:int = 6, h:int = 8, dropout:float = 0.1, d_ff:int = 2048)->Transformer:
  # Creating Embedding layers
  src_embed = InputEmbeddings(d_model, src_vocab_size)
  tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

  # Creating Positional Encoding layers
  src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
  tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

  # Creating EncoderBlocks
  encoder_blocks = []
  for _ in range(N):
    encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)

    # Combine layers into an EncoderBlock
    encoder_blocks = EncoderBlock(encoder_self_attention_block, feed_forward_block, dropout)
    encoder_blocks.append(encoder_blocks)

  # Creating DecoderBlocks
  decoder_blocks = []
  for _ in range(N):
    decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
    feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)

    # Combining layers into a DecoderBlock
    decoder_blocks = DecoderBlock(decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
    decoder_blocks.append(decoder_blocks)

  # Creating the Encoder and Decoder by using the EncoderBlocks and DecoderBlocks lists
  encoder = Encoder(nn.ModuleList(encoder_blocks))
  decoder = Decoder(nn.ModuleList(decoder_blocks))

  # Creating projection layer
  projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

  # Creating the transformer by combining everything above
  transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

  # Initialize the parameters
  for p in transformer.parameters():
    if p.dim() > 1:
      nn.init.xavier_uniform_(p)

  return transformer